`Author: Victor Radermecker | Date: 11/10/2021 | Project: Master Thesis`

This notebook computes the final demand estimation using the PPR and INDR ratios.

In [308]:
import json
import math
import os

# Importing local libraries
import sys

import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import *
from tqdm import tqdm

sys.path.insert(0, "../src")
from BrusselsMap import BrusselsMap
from GeoJsonHandler import GeoJsonHandler

pd.set_option("display.max_rows", 500)
pd.set_option('display.max_columns', 500)

In [309]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Demand Estimation Computation

#### Proximus data

In [310]:
prox = pd.read_csv("../data/final_demand_simulation/proximus_data_real.csv", sep=" ")
prox.head()

,originNB,destinationNB,regularTripSample,irregularTripSample
1,ALTITUDE 100,ALTITUDE 100,2897.762626,4181.745262
2,ALTITUDE 100,ANDERLECHT CENTRE - WAYEZ,15.557076,45.121344
3,ALTITUDE 100,ANNEESSENS,12.154057,37.092490
4,ALTITUDE 100,AUDERGHEM CENTRE,15.272227,31.149889
5,ALTITUDE 100,AVENUE LEOPOLD III,22.102622,40.685583


#### Convert to Energy 

In [311]:
dists = pd.read_csv("../data/final_demand_simulation/DemandClusters-ODMatrix-Km.csv").set_index(
    "origin"
)

# Convert number of trips into distance
dists_long = (
    dists.unstack()
    .reset_index()
    .rename(
        columns={"level_0": "destinationNB", "origin": "originNB", 0: "travelDistance"}
    )[["originNB", "destinationNB", "travelDistance"]]
)
prox = pd.merge(prox, dists_long, on=["originNB","destinationNB"], how="outer").fillna(0)

# Convert to Energy using IEA Assumption on EV's consumption (0.22kWh/km)
prox['regularDemand'] = prox['regularTripSample'] * prox['travelDistance']
prox['irregularDemand'] = prox['irregularTripSample'] * prox['travelDistance']
prox['totalDemand'] = prox['regularDemand'] + prox['irregularDemand']

prox['regularEnergyDemand'] = prox['regularDemand'] * 0.22 
prox['irregularEnergyDemand'] = prox['irregularDemand'] * 0.22 

#Aggregate data and save
prox = prox.groupby(by=['destinationNB'])[['regularTripSample','irregularTripSample', 'travelDistance', 'regularDemand', 'irregularDemand','totalDemand', 'regularEnergyDemand','irregularEnergyDemand']].sum().reset_index()
prox.rename(columns={'destinationNB':'NAME_FRE'}, inplace=True)
prox.to_csv('../data/final_demand_simulation/DemandEnergy.csv', index=False)

#### PPR Ratio

In [312]:
# Loading parking and population data
ppr = pd.read_csv("../data/final_demand_simulation/parkings_ratio.csv")
ppr.head()

,NAME_FRE,AREA,DENS,POP,AVG_HOUS,HOUSEHOLDS,NOT REGLEMENTED,RESERVED,REGLEMENTED,PRIVATE,PRIVATE/HOUS
0,CONSCIENCE,0.469490,17518.92,8225,2.33,3530,0,57,1260,551,0.156091
1,HELMET,0.718634,17717.47,12732,2.54,5013,0,81,2315,552,0.110114
2,VIEUX LAEKEN OUEST,0.499646,16781.39,8385,2.55,3288,0,90,996,310,0.094282
3,VIEUX LAEKEN EST,1.040450,18272.53,19012,2.61,7284,0,211,2858,664,0.091159
4,INDUSTRIE NORD,6.733601,261.99,1764,2.18,809,1277,26,248,218,0.269468


#### INDR Ratio

In [313]:
# Loading parking and population data
indr = pd.read_csv("../data/final_demand_simulation/INDR_distances.csv")
indr['ratio'] = indr['ratio'] / 100
indr.head()

,NAME_FRE,dist,ratio
0,CONSCIENCE,803.190416,0.198113
1,HELMET,852.081566,0.205969
2,VIEUX LAEKEN OUEST,665.214251,0.175941
3,VIEUX LAEKEN EST,1030.570800,0.234652
4,INDUSTRIE NORD,3110.748195,0.568927


### Merge datasets together

In [314]:
prox = pd.merge(prox, ppr[['NAME_FRE','PRIVATE/HOUS']], on="NAME_FRE", how="outer")
prox = pd.merge(prox, indr[['NAME_FRE','ratio']], on="NAME_FRE", how="outer")
prox.rename(columns={'ratio':'INDR_Ratio', 'PRIVATE/HOUS':'PPR_Ratio'}, inplace=True)
prox = prox.fillna(0)
prox.head(3)

,NAME_FRE,regularTripSample,irregularTripSample,travelDistance,regularDemand,irregularDemand,totalDemand,regularEnergyDemand,irregularEnergyDemand,PPR_Ratio,INDR_Ratio
0,ALTITUDE 100,10254.313281,17530.466331,1040.854,23064.265113,53951.988942,77016.254055,5074.138325,11869.437567,0.145925,0.192778
1,ANDERLECHT CENTRE - WAYEZ,12170.348991,20039.762646,1172.197,25316.225476,58414.817856,83731.043332,5569.569605,12851.259928,0.127411,0.262459
2,ANNEESSENS,6158.980624,14813.380991,852.715,14474.340595,39072.619201,53546.959797,3184.354931,8595.976224,0.066221,0.174679


#### Update the sectors.json file

In [315]:
# Create a GeoJson Handler
r_path = r"../data/final_demand_simulation/RBC_Neighborhoods_gps.json"
path = os.path.join(os.getcwd(), r_path)
sectors = GeoJsonHandler(path, "NAME_FRE")

In [316]:
dem = pd.read_csv(r"../data/final_demand_simulation/demographic.csv")
dem.head(3)

,NAME_FRE,AREA,DENS,POP,AVG_HOUS,HOUSEHOLDS
0,CONSCIENCE,0.469490,17518.92,8225,2.33,3530
1,HELMET,0.718634,17717.47,12732,2.54,5013
2,VIEUX LAEKEN OUEST,0.499646,16781.39,8385,2.55,3288


In [317]:
# Adding all trips from Altitude 100 
alt = pd.read_csv('../data/final_demand_simulation/ALTITUDE100.csv')
alt.rename(columns={'regularTripSample':'ALTITUDE100'}, inplace=True)
dem = pd.merge(dem, alt, on=["NAME_FRE"], how="left").fillna(3)

# Adding demographic data to sectors.json
attributes = [
    "DENS",
    "POP",
    "AVG_HOUS",
    "HOUSEHOLDS", 
    "ALTITUDE100"]

for att in attributes:
    sectors.add_property(dem, att)
    
# Adding proximus data to sectors.json
prox_attributes = list(prox.columns)[1:]

for att in prox_attributes:
    sectors.add_property(prox, att)

Successfully added DENS property to geodata.
Successfully added POP property to geodata.
Successfully added AVG_HOUS property to geodata.
Successfully added HOUSEHOLDS property to geodata.
Successfully added ALTITUDE100 property to geodata.
Successfully added regularTripSample property to geodata.
Successfully added irregularTripSample property to geodata.
Successfully added travelDistance property to geodata.
Successfully added regularDemand property to geodata.
Successfully added irregularDemand property to geodata.
Successfully added totalDemand property to geodata.
Successfully added regularEnergyDemand property to geodata.
Successfully added irregularEnergyDemand property to geodata.
Successfully added PPR_Ratio property to geodata.
Successfully added INDR_Ratio property to geodata.


In [318]:
# Save geoJson for future use
r_path = r"../data/final_demand_simulation/sector.json"
o_path = os.path.join(os.getcwd(), r_path)

sectors.save_json(o_path)

Successfully saved geojson.


# Some visualizations here

Possible elements to plot:
- 'AREA'
- 'DENS'
- 'POP'
- 'AVG_HOUS'
- 'HOUSEHOLDS'
- 'regularTripSample'
- 'irregularTripSample'
- 'travelDistance'
- 'regularDemand'
- 'irregularDemand'
- 'totalDemand'
- 'regularEnergyDemand'
- 'irregularEnergyDemand'
- 'PPR_Ratio'
- 'INDR_Ratio'

In [319]:
#Load geodata
r_path = r"../data/final_demand_simulation/sector.json"
path = os.path.join(os.getcwd(), r_path)
sectors = GeoJsonHandler(path, "NAME_FRE")

#### Raw Number of Regular Trips

In [320]:
mymap = BrusselsMap(sectors, 800, 800, tile="stamenwatercolor")

attribute = 'regularTripSample'
legend = f"Choropleth Map of Brussels showing the {attribute}"
nbr_steps = 4
colormap_type = "linear"
mymap.add_choropleth_style2(sectors.gdf[["NAME_FRE", attribute]], legend, nbr_steps, colormap_type)
mymap.map

#### Plot Trips for Altitude 100

In [330]:
colormap = cm.LinearColormap(
    ["white", "yellow", "darkorange", "red", "darkred", "black"], index=[0, 0.1, 0.25, 0.5, 0.75, 1.0]
)
colormap

In [348]:
colormap = cm.LinearColormap(
    ["white", "yellow", "darkorange", "red", "darkred", "black"], vmin=0, vmax=3000
)
colormap = colormap.to_step(
    n=3,
    data=dem["ALTITUDE100"],
    method='quantiles',
    round_method='float'
)
colormap

In [408]:
colormap = cm.LinearColormap(
    ["white", "beige", "lightgray", "yellow", "darkorange", "red", "darkred", "black"], index=[0, 5, 15, 30, 60, 200, 1000, 3000], vmin=1, vmax=3000
)
colormap

In [409]:
mymap = BrusselsMap(sectors, 800, 800, tile="stamenwatercolor")

attribute = 'ALTITUDE100'
legend = f"Choropleth Map of Brussels showing the {attribute}"
nbr_steps = 4
colormap_type = "manual"
mymap.add_choropleth_style2(sectors.gdf[["NAME_FRE", attribute]], legend, nbr_steps, colormap_type, colormap)
mymap.map